<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/072_Request_Specific_Dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Dependency Injection via ActionContext

### ✅ Core Concepts

* **Dependency Injection via `ActionContext`**
  Tools should *not* directly access global or hardcoded resources like memory, auth tokens, or APIs. Instead, those resources are *injected* through `ActionContext`, keeping tools **modular, testable, and reusable**.

* **Session-Scoped Context**
  Each agent run (or request) may require its own:

  * Memory
  * Authentication token
  * LLM instance
  * Runtime config

  All of this is encapsulated in `ActionContext`, like a delivery box of everything a tool needs.

---

## 🧰 What to Focus On in the Code

### 🔐 Auth Token Injection

```python
auth_token = action_context.get("auth_token")
```

This line shows how a tool accesses an *external, per-request secret* **without hardcoding** or global configuration.

> ✅ Focus: How this protects security, enables per-user customization, and avoids brittle code.

---

### 🧩 Tool Independence

The tool doesn't know where `auth_token` came from or how it's stored — it just uses it. That's good software design: **low coupling, high cohesion**.

---

### 🏗️ Context Construction in Agent

```python
action_context = ActionContext({
    'memory': memory,
    'llm': self.generate_response,
    **action_context_props
})
```

This is where the *agent constructs the environment* the tools will run in. It assembles memory, LLM, and any per-request context (e.g., user token, timezone, etc.).

> ✅ Focus: The agent becomes a **provider of context**, not a hardcoded dependency.

---

### 🧪 Real-World Use Case

Imagine:

* A user logs in via OAuth.
* You need to fetch or update data *on their behalf*.
* Instead of embedding user secrets in tools, you pass them safely through `ActionContext`.

This pattern is used across scalable systems (think FastAPI dependencies, Flask `request`, React context, etc.)

---

## 🏁 Summary Takeaways

* `ActionContext` lets you build **clean**, **modular**, and **secure** AI tools.
* You can inject anything: memory, LLMs, auth tokens, configs, experiment flags, etc.
* Agents become orchestrators of resources — not owners of everything.




In [ ]:
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext,
                        review_id: str,
                        status: str) -> dict:
    """Update the status of a code review in the project system."""
    # Get the authentication token for this specific request
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")

    # Make authenticated request
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )

    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")

    return {"status": "updated", "review_id": review_id}




### 1. **Contextual Dependency Access (Not Hardcoding!)**

```python
auth_token = action_context.get("auth_token")
```

✅ **Why it matters:**

* Instead of embedding secrets or assuming global state, this tool **expects the token to be provided at runtime**.
* This is **dependency injection in practice** — it's flexible, secure, and testable.
* The tool doesn’t care *where* the token came from — it just asks the context for it.

---

### 2. **Fails Fast if Context is Missing**

```python
if not auth_token:
    raise ValueError("Authentication token not found in context")
```

✅ **Why it matters:**

* This makes the tool **safe and predictable**.
* If a developer or orchestrator forgets to provide the token, the tool tells you exactly what's wrong — no silent failures or mysterious bugs.

---

### 3. **Encapsulated External API Call**

```python
response = requests.post(...)
```

✅ **Why it matters:**

* This is a **clear separation of concerns**: the tool wraps a real-world side effect (updating a remote system), but all that complexity is hidden from the agent.
* It doesn’t expose how the API works to the rest of the system — just that *you can update a review status*.

---

### 4. **Return Value is Structured**

```python
return {"status": "updated", "review_id": review_id}
```

✅ **Why it matters:**

* Structured return values make orchestration easier.
* Agents and downstream tools can act on predictable keys like `"status"` and `"review_id"`.

---

## 🧠 What You Should Be Learning

* How to **inject secure, request-specific data** (like auth tokens) into tools without hardcoding.
* How to **fail clearly** when required context is missing.
* How to make tools **stateless and composable**, so they can run in many different environments or orchestrations.
* How to **cleanly wrap external APIs** inside reusable, testable tools.




What you're seeing here with `action_context.get("auth_token")` is a **more dynamic, runtime-safe version** of the same principle that traditional software development uses with `.env` files or config management.

Let’s compare the ideas side-by-side:

---

### ✅ Traditional Development

| Concept                  | Example                                   | Purpose                                      |
| ------------------------ | ----------------------------------------- | -------------------------------------------- |
| **Environment Variable** | `.env` file or `os.environ["AUTH_TOKEN"]` | Stores secrets/config outside of source code |
| **Config Injection**     | DI frameworks (like in Flask or Spring)   | Injects API keys, DB connections, etc.       |
| **Fail Fast**            | Raise error if env var is missing         | Prevents misuse in dev/prod                  |

---

### ✅ Agent Tooling with `ActionContext`

| Concept               | Example                                 | Purpose                                                 |
| --------------------- | --------------------------------------- | ------------------------------------------------------- |
| **Context Injection** | `action_context.get("auth_token")`      | Passes per-request secrets or tokens without hardcoding |
| **Dynamic Isolation** | Context can vary by session or user     | Supports multiple concurrent users/sessions securely    |
| **Error Handling**    | Raises `ValueError` if token is missing | Ensures tool is only used when properly configured      |

---

### 🔐 Why This is *Better* for Agents

* Agents run **dynamically per task**, not on a single server instance — so `.env` files alone aren’t flexible enough.
* You might want to **pass different API tokens per user**, session, or role — ActionContext allows that without global state.
* Tools become **fully decoupled and testable** — you can pass in mock tokens during testing, or switch environments effortlessly.






## 🔁 Recap: First Agent – *ActionContext with Per-Request Token*

```python
auth_token = action_context.get("auth_token")
```

### ✅ What It Shows:

* This agent **expects the token to be provided at runtime**, via the `ActionContext`.
* It allows for **flexible, per-user, or per-session access**.
* **Nothing is hardcoded** — the tool has no knowledge of where the token comes from.

This is similar to **dependency injection** — the agent tool only cares that it receives what it needs when it runs. This keeps the tool **testable, reusable, and safe**.

---

## ⛓️ Now Let’s Contrast With the Second Agent (From the Lecture)

Look for this kind of pattern in the second agent:

```python
def __init__(self, github_token: str):
    self.github_token = github_token
```

Or:

```python
auth_token = self.github_token  # Already stored in agent state
```

### 🔒 What This Approach Shows:

* The second agent **holds onto the dependency** (e.g., token or secret) **at construction time**.
* This token becomes part of the **agent’s persistent configuration**.
* This is useful when:

  * The agent is tied to a **single system account or integration**.
  * You want to **reuse the token** across many tasks without injecting it repeatedly.
  * The agent is **long-lived** and not user-specific.

---

## 🔍 Summary: What's the Core Difference?

| Feature          | First Agent                 | Second Agent                   |
| ---------------- | --------------------------- | ------------------------------ |
| Token passed via | `ActionContext`             | Agent constructor or config    |
| Scope of token   | Per-request, per-user       | Shared or persistent           |
| Flexibility      | ✅ More dynamic              | 🚫 Less dynamic                |
| Testability      | ✅ Easy to test              | ⚠️ Harder (stateful)           |
| Isolation        | ✅ Safe for multi-user       | ⚠️ Risky if reused             |
| Use case         | Multi-user, ephemeral tools | Dedicated, system-level agents |

---

## 🧠 So What Should You Learn?

* Use **ActionContext injection** when building modular, secure, and reusable **tools**.
* Use **persistent agent config** only when you **need global access** or are building a long-lived agent with fixed credentials.
* Think of it like **stateless vs. stateful design** in services.

---




## ✅ Is Agent 2 *Better*, *Worse*, or Just *Different*?

**Short Answer:**
It’s **not about better or worse** — it’s about **tradeoffs** and choosing the right tool for the job.

Agent 1 and Agent 2 are solving *different architectural problems*:

| Feature                     | Agent 1 (Request-specific)               | Agent 2 (Persistent config)                     |
| --------------------------- | ---------------------------------------- | ----------------------------------------------- |
| **How it gets credentials** | Injected per request via `ActionContext` | Passed once at setup and stored in the agent    |
| **Scope of use**            | Flexible: per-user, per-session          | Fixed: shared access, often system-wide         |
| **Security model**          | Least privilege per use                  | Long-lived access token                         |
| **Reusability**             | Highly reusable tool                     | Tightly coupled to one specific service/account |
| **Best for**                | Multi-tenant apps, sandboxed tools       | Dedicated integrations, internal services       |

---

## 🧠 When to Use **Agent 1** (Request-Specific Injection):

* You are building a **multi-user system** (e.g. a platform where each user has different credentials).
* You want to **rotate tokens**, use short-lived credentials, or authenticate per session.
* You want **maximum testability** and **low coupling**.
* You're designing **stateless tools** (no internal memory of credentials or identity).

✅ Example use cases:

* Connecting to a user's Google Calendar during an onboarding flow.
* Making API calls on behalf of different users with their own access tokens.
* Isolating access to prevent cross-user data leakage.

---

## 🧠 When to Use **Agent 2** (Persistent Config):

* The agent represents a **single service identity** (e.g. your company’s GitHub bot).
* You want to **avoid repeatedly injecting credentials**.
* The agent **performs tasks on its own behalf**, not as a proxy for someone else.
* You’re dealing with **background agents**, system tasks, or integrations you fully control.

✅ Example use cases:

* A CI/CD bot that leaves comments on pull requests.
* A documentation assistant that updates a wiki.
* An analytics agent that periodically reads from a database with fixed credentials.

---

## 🎯 Final Takeaway:

> Agent 1 = *Stateless, modular, dynamic.*
> Agent 2 = *Stateful, focused, consistent.*

In real-world systems, you’ll likely use **both** patterns:

* Modular tools (Agent 1) for most tasks.
* Dedicated agents (Agent 2) for things like long-term integrations or internal automation.


In [ ]:
def run(self, user_input: str, memory=None, action_context_props=None):
    """Execute the agent loop."""
    memory = memory or Memory()

    # Create context with all necessary resources
    action_context = ActionContext({
        'memory': memory,
        'llm': self.generate_response,
        # Request-specific auth
        **action_context_props
    })

    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)

        if self.should_terminate(action_context, response):
            break

...
# Run the agent and create custom context for the action to
# pass to tools that need it
some_agent.run("Update the project status...",
               memory=...,
               # Pass request-specific auth token
               action_context_props={"auth_token": "my_auth_token"})




## 🔍 **What to Focus On in the Second Agent**

### 1. **Dynamic Context Construction**

```python
action_context = ActionContext({
    'memory': memory,
    'llm': self.generate_response,
    **action_context_props
})
```

**What’s happening?**

* The `ActionContext` is being **dynamically assembled** at runtime using:

  * `memory`: the conversation history.
  * `llm`: a reference to the agent’s language model function.
  * `action_context_props`: arbitrary, task-specific parameters (like auth tokens, session IDs, etc.)

✅ **Why it matters:**
This makes your tools *extremely modular* and *testable*, because the agent doesn’t hardcode access to things like auth tokens — they’re passed in cleanly.

---

### 2. **Passing Session-Scoped Resources**

```python
action_context_props={"auth_token": "my_auth_token"}
```

**What’s happening?**

* The caller of `run()` is injecting session/request-specific resources like an authentication token.
* These will **automatically be available** inside tools via `action_context.get("auth_token")`.

✅ **Why it matters:**
You’re not leaking global state. Each run can have its own credentials, context, or resources.

---

### 3. **Custom Agent Loop**

```python
while True:
    prompt = ...
    response = ...
    result = ...
    
    if self.should_terminate(...):
        break
```

**What’s happening?**
This is a **customizable agent loop** that lets you:

* Construct a prompt based on memory and goals
* Interpret the LLM's output
* Take actions (or not)
* Continue or terminate

✅ **Why it matters:**
You're building a **controlled, auditable agent runtime**, not blindly prompting a GPT model. You own the orchestration logic.

---

### 4. **Agent Becomes a Platform**

```python
some_agent.run("Update the project status...",
               memory=...,
               action_context_props={"auth_token": "my_auth_token"})
```

**What’s happening?**
You’re calling the agent like a **service**, supplying memory + any dependencies it needs. This turns the agent into a:

* **Composable component**
* **Reentrant function**
* **Platform plugin**

✅ **Why it matters:**
This pattern allows **other agents**, **pipelines**, or **human workflows** to reuse this agent in predictable, repeatable ways.

---

## 🎯 Big Picture Takeaway

You're seeing a **clean architecture for running agents as dependency-injected services**.
Focus on:

* How dependencies are passed in (`action_context`)
* Why state is isolated per session
* How this decouples the agent from global config or fragile imports
* How this allows the same agent to work in *dev, test, and prod* without changes




## 🔄 Part 1: **How `ActionContext` is Dynamically Assembled (and How That Differs)**

### 🔧 **What you've likely done before:**

In many earlier examples or tutorials, you might have built agents like this:

```python
# Not dynamic — hardcoded or globally configured
memory = Memory()
llm = OpenAI()

result = some_tool(memory=memory, llm=llm, ...)
```

This tightly **couples the tool or agent to its environment**:

* It assumes a specific memory.
* It assumes a specific LLM.
* It often uses globals or shared state (e.g. hardcoded API keys or tokens).

---

### ✅ **How this new version differs:**

Here, we’re **constructing `ActionContext` dynamically at runtime**:

```python
action_context = ActionContext({
    'memory': memory,
    'llm': self.generate_response,
    **action_context_props
})
```

This gives you **dependency injection**:

* You can plug in different `memory`, `llm`, or `auth_token` at **runtime**.
* No hardcoded resources.
* Each agent run has its **own context** with **isolated state**.

This makes your tools and agents:

* **Reusable** (they don’t care where the memory or token came from).
* **Testable** (you can pass mocks).
* **Safer** (no accidental global state leaks).

---

## 🔁 Part 2: **Why the While Loop?**

### 🧠 What the loop is doing:

```python
while True:
    prompt = self.construct_prompt(...)
    response = self.prompt_llm_for_action(...)
    result = self.handle_agent_response(...)
    
    if self.should_terminate(...):
        break
```

This is the **core agent loop**.

#### 📥 Why do we need a loop?

* Agents often don’t complete a task in one step.
* The LLM might decide it needs to:

  * Call a tool
  * Ask a follow-up question
  * Perform another round of reasoning
* Each loop is a **cycle of planning, acting, reflecting**.

Think of it like an async control center:

* Construct a prompt → Send to LLM → Handle result → Decide next step → Repeat

---

## ✨ Summary: Why This Matters

| Concept                        | Before              | Now (with ActionContext + loop)     |
| ------------------------------ | ------------------- | ----------------------------------- |
| Resource management            | Hardcoded or global | Injected at runtime                 |
| Reusability                    | Low                 | High — tools/agents are pluggable   |
| Testability                    | Difficult           | Easy — you can mock memory/LLM/etc. |
| Execution model                | One-shot            | Multi-turn, stateful, controlled    |
| Scalability to teams/pipelines | Fragile             | Cleanly orchestrated and modular    |






### 🔌 **Tools and components are "plug-and-play":**

* Tools don't hardcode their dependencies.
* You can swap in:

  * Different **LLMs**
  * Different **memory providers**
  * Context-specific data like `auth_token`, `user_id`, or `project_id`

→ Makes it easy to reuse tools across agents or environments (dev, prod, test).

---

### 🔁 **The agent becomes a controlled orchestrator:**

* The **loop** gives you multiple reasoning + action steps.
* Tools can be called as needed, in order.
* The agent can pause, wait for human feedback, or reflect between steps.

→ Makes it robust for complex workflows (e.g. code generation, research, planning).

---

### ⚙️ **Easy to integrate into larger pipelines:**

* Tools and agents now behave like **components** in a system.
* You can:

  * Chain agents together
  * Delegate tasks between them
  * Swap or upgrade components without rewriting everything

---

### 🧪 Bonus: Easier to test

* You can **mock the `ActionContext`** for unit tests:

  ```python
  ctx = ActionContext({'memory': FakeMemory(), 'llm': fake_llm})
  ```

→ Great for building production-ready systems with confidence.


